Define some required variables, parents, and functions:

In [ ]:
var('k t z r')
assume(k, 'integer')

In [ ]:
def upsilon(z):
    return (1 - sqrt(1 - 4*z))/(2*z) - 1
def chi(k):
    return 2*pi*I*k/log(2)
PR_R = PolynomialRing(ZZ, "R").fraction_field()
R = PR_R("R")
AR.<n> = AsymptoticRing('QQ^n * n^QQ * log(n)^QQ', QQ)

In [ ]:
AR_Z.<Z> = AsymptoticRing('Z^QQ * log(Z)^QQ', PR_R)
AR_T.<T> = AsymptoticRing('T^QQ * log(T)^QQ', SR.subring(accepting_variables=[k]))

In [ ]:
zl = 1/4*(1-1/Z)
assert 1-4*zl == 1/Z

### Compactification degree

Generating function for paths with compactification degree $\leq r$:

In [ ]:
def L(r, z):
    return sum(4^(j+1) * upsilon(z)^(2^j)/(1 + upsilon(z)^(2^j))^2 for j in srange(0, r+1))

Expansions for small values of $r$:

In [ ]:
L(0, z).taylor(z, 0, 20)

In [ ]:
L(1, z).taylor(z, 0, 20)

In [ ]:
L(2, z).taylor(z, 0, 20)

In [ ]:
L(3, z).taylor(z, 0, 20)

#### Theorem 3; Expectation

Mellin transform:

In [ ]:
g1(s) = 4 * gamma(s) * zeta(s-1) * exp(log(2)*(2-s)) / (1 - exp(log(2)*(2-s)))

In [ ]:
g1_residue_nonfluc = fast_callable((g1(s)*t^-s).residue(s==2) + 
                                   (g1(s)*t^-s).residue(s==0), vars=[t])(1/T) + O(1/T^2)
g1_residue_nonfluc.map_coefficients(lambda x: x.simplify().expand()).show()

In [ ]:
def t_in_z(z):
    return -log(upsilon(z))
t_in_z(zl + O(Z^-4))

In [ ]:
g1_expansion_nonfluc = g1_residue_nonfluc.subs(T=1/t_in_z(zl+O(Z^-4)))
g1_expansion_nonfluc = g1_expansion_nonfluc.map_coefficients(lambda ex: ex.expand().simplify())
g1_expansion_nonfluc.show()

In [ ]:
rdeg_expectation_nonfluc = g1_expansion_nonfluc._singularity_analysis_(n, 1/4, precision=3) / AR(4)^n
rdeg_expectation_nonfluc.show()

Residue at $2 + \chi_k$:

In [ ]:
g1_residue_fluc_k = fast_callable((g1(s) * t^-s).residue(s==2 + chi(k)), 
                                  vars=[t, k])(1/T, k)
g1_residue_fluc_k.show()

In [ ]:
g1_expansion_fluc_k = g1_residue_fluc_k.subs(T=1/t_in_z(zl+O(Z^-2)))
g1_expansion_fluc_k = g1_expansion_fluc_k.map_coefficients(lambda ex: ex.subs({exp((2+chi(k))*log(1/2)):1/4}))
g1_expansion_fluc_k.show()

In [ ]:
rdeg_expectation_fluc_k = g1_expansion_fluc_k._singularity_analysis_(n, 1/4, precision=1) / 4^n
rdeg_expectation_fluc_k = rdeg_expectation_fluc_k.exact_part() + O(n^-1)
rdeg_expectation_fluc_k.show()

#### Theorem 3; Variance

Mellin transform:

In [ ]:
g2(s) = 4 * gamma(s) * zeta(s-1) * (1 + exp(log(2)*(2-s))) * exp(log(2)*(2-s)) / (1 - exp(log(2)*(2-s)))^2

In [ ]:
zetaderivrule = {stieltjes(1):
                 2*(zetaderiv(2, 0)+1/2*(log(pi)+log(2))^2 - 1/2*stieltjes(0)^2+pi^2/24)}

In [ ]:
g2_residue_nonfluc = fast_callable((g2(s)*t^-s).residue(s==2) 
                                   + (g2(s)*t^-s).residue(s==0), vars=[t])(1/T) + O(1/T^2)
g2_residue_nonfluc = g2_residue_nonfluc.map_coefficients(
    lambda ex: ex.subs(zetaderivrule).canonicalize_radical().expand())
g2_residue_nonfluc.show()

In [ ]:
g2_expansion_nonfluc = g2_residue_nonfluc.subs(T=1/t_in_z(zl+O(Z^-3))) + O(Z^0)
g2_expansion_nonfluc = g2_expansion_nonfluc.map_coefficients(
    lambda ex: ex.canonicalize_radical().expand())
g2_expansion_nonfluc.show()

In [ ]:
rdeg_moment2_nonfluc = g2_expansion_nonfluc._singularity_analysis_(n, 1/4, precision=5) / 4^n
rdeg_moment2_nonfluc = rdeg_moment2_nonfluc.map_coefficients(lambda ex: ex.canonicalize_radical().expand())
rdeg_moment2_nonfluc.show()

Residue at $2 + \chi_k$:

In [ ]:
(g2(s)*t^-s).residue(s==2+chi(k)).expand().show()

In [ ]:
g2_residue_fluc_k = fast_callable((g2(s)*t^-s).residue(s==2+chi(k)).expand(), vars=[t,k])(1/T, k)
g2_residue_fluc_k.show()

In [ ]:
g2_expansion_fluc_k = g2_residue_fluc_k.subs(T=1/t_in_z(zl+O(Z^-2)))
g2_expansion_fluc_k = g2_expansion_fluc_k.map_coefficients(lambda ex: ex.canonicalize_radical().expand())
g2_expansion_fluc_k.show()

General residue:

In [ ]:
rdeg_moment2_fluc_k = g2_expansion_fluc_k._singularity_analysis_(n, 1/4, precision=2) / 4^n
rdeg_moment2_fluc_k = rdeg_moment2_fluc_k.map_coefficients(lambda ex: ex.expand().subs({factorial((2*I*pi*k+log(2))/log(2)):gamma(chi(k) + 2)}))
rdeg_moment2_fluc_k.show()

In [ ]:
g2_residue_fluc_k = (g2(s)*t^-s).residue(s==2+chi(k)).expand()
g2_residue_fluc_k.show()

Singular expansion (with an error of $O(\log(1 - 4z))$):

In [ ]:
(g2_residue_fluc_k.subs(t=-log(upsilon(z))).subs(z=(1-z)/4).taylor(z, 0, 0).coefficient(z, -1)/z).expand().subs(z=1-4*z).show()